# Gathering Data
Take your data and upload to Cloudant.

# Converting Data into CSV
The data in the database must be converted into a form that can be used to train the models for entity extraction and natural language classification. To do this, run the script below. Please edit the database connections so that they point to your database where all your data is stored.

In [ ]:
import cloudant
import csv

SERVER = 'https://1790ef54-fcf2-4029-9b73-9000dff88e6e-bluemix.cloudant.com' #Replace with your server URL
DATABASE = 'amazon_data' #Replace with the name of the database
USERNAME = '1790ef54-fcf2-4029-9b73-9000dff88e6e-bluemix' #Replace with the username from your credentials for the NLC
PASSWORD = '5beb3f8b9f95586542e3d9c5acfb0c52832252432623e534d4e88b12fad29638' #Replace with the password from your credentials for the NLC
DESIGN = '' #replace with the name of the design document that contains the view. This should be of the form '_design/XXXX'
VIEW = 'names/namesAndAsin' #Replace with the view from your database to poll, this should take the form of view_file/view and should return the text to classify as the value field and what you would like to call it as the key
DESTINATION = 'out.csv' #Replace with correct name for output file (NOTE must be *.csv)

server = cloudant.client.Cloudant(USERNAME,PASSWORD,url=SERVER)
server().connect
db = server[DATABASE]
query = db.get_view_result(DESIGN,VIEW)
file = open(DESTINATION, 'wb')
writer = csv.writer(file)


for q in query:
    print q[0]
    if 'key' in q[0] and q[0]['key'] != None:
        title = q[0]['key']
    else:
        title = "No Title"
    if 'value' in q[0] and q[0]['value'] != None:
        text = q[0]['value']
    else:
        text = "No Text"
    writer.writerow([title,text])

file.close()

# Import Data CSV into Watson Knowledge Studio
The CSV that is created at the end of the script must now be imported into Watson Knowledge Studio (WKS). Each row in the CSV will be treated as a separate document and will be organized in WKS. The documents then need to be annotated with entities and relationships. Coreference is also done here. A strict guideline is very helpful when doing this.

After the annotations are done and the model is trained, it needs to be exported into Alchemy Language using an API key.

In [ ]:
import token_replacement as t
import csv

read = open('ground_truth_layer1.csv','rb') ##replace with correct filenames
write = open('ground_truth_layer1_replace.csv','wb')     ##replace with correct filenames

reader = csv.reader(read)
writer = csv.writer(write)

for row in reader:
    token = t.token_replacement(row[0])
    writer.writerow([token,row[1]])

read.close()
write.close()